In [51]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [52]:
# Import necessary libraries
import os
import torch
import sys
import torch.nn as nn
from torch.utils.data import DataLoader
import torchvision
import pandas as pd
import numpy as np
import cv2
from albumentations.pytorch import ToTensorV2
import albumentations as A


# Navigate to the directory containing the images and weights
%cd /content/drive/MyDrive/Defence/Data/Dataset/Tomato/leaf curl
!ls

/content/drive/MyDrive/Defence/Data/Dataset/Tomato/leaf curl
'0tomato_train_leaf curl.JPG'	  '1711tomato_train_leaf curl.JPG'  '406tomato_train_leaf curl.JPG'
'0tomato_valid_leaf curl.JPG'	  '1712tomato_train_leaf curl.JPG'  '406tomato_valid_leaf curl.JPG'
'1000tomato_train_leaf curl.JPG'  '1713tomato_train_leaf curl.JPG'  '407tomato_train_leaf curl.JPG'
'1001tomato_train_leaf curl.JPG'  '1714tomato_train_leaf curl.JPG'  '407tomato_valid_leaf curl.JPG'
'1002tomato_train_leaf curl.JPG'  '1715tomato_train_leaf curl.JPG'  '408tomato_train_leaf curl.JPG'
'1003tomato_train_leaf curl.JPG'  '1716tomato_train_leaf curl.JPG'  '408tomato_valid_leaf curl.JPG'
'1004tomato_train_leaf curl.JPG'  '1717tomato_train_leaf curl.JPG'  '409tomato_train_leaf curl.JPG'
'1005tomato_train_leaf curl.JPG'  '1718tomato_train_leaf curl.JPG'  '409tomato_valid_leaf curl.JPG'
'1006tomato_train_leaf curl.JPG'  '1719tomato_train_leaf curl.JPG'  '40tomato_train_leaf curl.JPG'
'1007tomato_train_leaf curl.JPG'  '171tomato

In [53]:
import torch.utils.data as data

class Mydataset(data.Dataset):
    def __init__(self, img_paths, labels, transform):
        self.imgs = img_paths
        self.labels = labels
        self.transforms = transform

    def __getitem__(self, index):
        img = self.imgs[index]
        label = self.labels[index]
        labels = int(label)
        data = self.transforms(image=img)
        return data['image'], labels

    def __len__(self):
        return len(self.imgs)

class Mydataset_infer(data.Dataset):
    def __init__(self, img_paths, transform):
        self.imgs = img_paths
        self.transforms = transform

    def __getitem__(self, index):
        img_path_here = self.imgs[index]
        img = cv2.imread(img_path_here)[:,:,::-1]
        data = self.transforms(image=img)
        return img_path_here, data['image']

    def __len__(self):
        return len(self.imgs)

def for_train_transform():
    train_transform = A.Compose([
        A.RandomRotate90(),
        A.Flip(p=0.5),
        A.Normalize(
            mean=[0.485, 0.456, 0.406],
            std= [0.229, 0.224, 0.225],
            max_pixel_value=255.0,
            p=1.0
        ),
        ToTensorV2()], p=1.)
    return train_transform

test_transform = A.Compose([
    A.Normalize(
        mean=[0.485, 0.456, 0.406],
        std= [0.229, 0.224, 0.225],
        max_pixel_value=255.0,
        p=1.0
    ),
    ToTensorV2()], p=1.)


In [ ]:
# Set paths and other configurations
imgs_infer_path = '/content/drive/MyDrive/Defence/Data/Dataset/Tomato/verticulium wilt'
weight_path = '/content/drive/MyDrive/Defence/Data/Dataset/Tomato/verticulium wilt/Active Learning.pth'
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

# List images for inference
name_list = os.listdir(imgs_infer_path)
infer_images2 = [''.join([imgs_infer_path, '/', i]) for i in name_list]

# Create dataset and dataloader
test_ds = Mydataset_infer(infer_images2, test_transform)
test_dl = DataLoader(test_ds, batch_size=1, shuffle=False, pin_memory=False, num_workers=4,)

# Define loss criterion
criterion = nn.MSELoss().to('cuda')

def main():
    # Initialize model and load weights
    model = torchvision.models.resnet34(pretrained=False)
    model.fc = nn.Linear(in_features=512, out_features=8, bias=True)
    model = model.to('cuda')
   # model.load_state_dict(torch.load(weight_path))
    model.eval()

    loss_list, name_list = [], []

    with torch.no_grad():
        for batch_idx, (name, imgs) in enumerate(test_dl):
            sys.stdout.write('\r%d/%s' % (batch_idx + 1, len(test_dl)))
            name_pic = name[0].split('/')[-1]
            imgs = imgs.to('cuda').float()
            masks_pred = model(imgs)
            out1 = model(imgs.flip([-1]))
            out2 = model(imgs.flip([-2]))
            out3 = model(imgs.flip([-1, -2]))
            masks_pred = torch.softmax(masks_pred,1)
            out1, out2, out3 = torch.softmax(out1,1), torch.softmax(out2,1), torch.softmax(out3,1)
            average = (out1 + out2 + out3 + masks_pred) / 4
            loss = criterion(torch.cat((out1, out2, out3, masks_pred),0),
                             torch.cat((average, average, average, average),0)).item()
            loss_list.append(loss)
            name_list.append(name_pic)

    # Save results to CSV and sort them by loss
    df = pd.DataFrame({
        'image_name': name_list,
        'loss': loss_list
    })
    df = df.sort_values('loss', ascending=False)
    df.to_csv('/content/drive/MyDrive/Defence/Code/Tomato/AL_verticulium wilt.csv', index=False)

if __name__ == '__main__':
    main()


/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:558: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
/usr/lib/python3

2021/3864

In [ ]:
%pwd

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Defence/Code/Tomato/AL_verticulium wilt.csv')

In [ ]:
df.head(50)